<a href="https://colab.research.google.com/github/ILY-OGAMA/Market-Basket-Analysis/blob/main/Market_Basket_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Basket Analysis
## Association Rule Mining
###  Market Basket Analysis (MBA) is an accidental transaction pattern that purchasing some products will affect the purchasing of other products. MBA is used to predict what products that customer interested in.












In [ ]:
# install libraries that we will need
! pip install xlrd==1.2.0
! pip install psycopg2
! pip install fsspec

In [ ]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline
import psycopg2

# Import dataset
## We use a transaction data from an online retail store

In [5]:
from google.colab import files
uploaded = files.upload()

Saving online_retail.xlsx to online_retail.xlsx


In [6]:
# get the data set we need 
df_1 = pd.read_excel("online_retail.xlsx", sheet_name = "Year 2010-2011")
df_1

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [7]:
# get the data set
df_2 = pd.read_excel("online_retail.xlsx")
df_2

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


# Process the data 
1. Combine the two data sets
2. Handle null values
3. Use positive 'Quantity' values


In [8]:
# combine the two data sets into one 
data = [df_1, df_2]
retail = pd.concat(data)
retail

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom


In [9]:
# drop all null values
retail = retail.dropna()
retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824364 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      824364 non-null  object        
 1   StockCode    824364 non-null  object        
 2   Description  824364 non-null  object        
 3   Quantity     824364 non-null  int64         
 4   InvoiceDate  824364 non-null  datetime64[ns]
 5   Price        824364 non-null  float64       
 6   Customer ID  824364 non-null  float64       
 7   Country      824364 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 56.6+ MB


In [10]:
# in this data set quantity shows the number of items that are bought in each transaction 
# where there is a cancellation on a particular transaction it is datificated as a negative
# thats why we aren't going to use them

retail_plus = retail[retail['Quantity']>0]
retail_plus.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 805620 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      805620 non-null  object        
 1   StockCode    805620 non-null  object        
 2   Description  805620 non-null  object        
 3   Quantity     805620 non-null  int64         
 4   InvoiceDate  805620 non-null  datetime64[ns]
 5   Price        805620 non-null  float64       
 6   Customer ID  805620 non-null  float64       
 7   Country      805620 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 55.3+ MB


# Create the Basket data 
The basket will contain the quantity of each item bought per transaction

In [11]:
# create a basket data while using transactions from UK only since over 90% of the
# transactions are from the 

basket = (retail_plus[retail_plus['Country']=='United Kingdom'].groupby(['Invoice','Description'])
          ['Quantity'].sum().unstack().reset_index().fillna(0).set_index('Invoice'))
basket

Description,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,Bank Charges,CAMPHOR WOOD PORTOBELLO MUSHROOM,CHERRY BLOSSOM DECORATIVE FLASK,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
489438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Encode the data

In [12]:
# encode the basket data into a binary data that shows whether an item is bought (1)
# or not (0)
def encode_units(x):
  if x <= 0 :
    return 0
  if x >= 1 :
    return 1

basket_encode_plus = basket.applymap(encode_units)
basket_encode_plus  

Description,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,Bank Charges,CAMPHOR WOOD PORTOBELLO MUSHROOM,CHERRY BLOSSOM DECORATIVE FLASK,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Filter the transactions

In [13]:
# filter the transaction to bought more than 1 itemss only
basket_filter = basket_encode_plus[(basket_encode_plus > 0).sum(axis = 1) >=2]
basket_filter

Description,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,Bank Charges,CAMPHOR WOOD PORTOBELLO MUSHROOM,CHERRY BLOSSOM DECORATIVE FLASK,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Apply the apriori algorithim

In [ ]:
# apply the apriori algorithim
! pip install mlxtend


In [15]:
from mlxtend.frequent_patterns import apriori

In [33]:
frequent_items = apriori(basket_filter, min_support= 0.01,
                         use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_items['length']= frequent_items['itemsets'].apply(lambda x: len(x))
frequent_items

,support,itemsets,length
0,0.151016,(WHITE HANGING HEART T-LIGHT HOLDER),1
1,0.091207,(REGENCY CAKESTAND 3 TIER),1
2,0.080947,(ASSORTED COLOUR BIRD ORNAMENT),1
3,0.076823,(JUMBO BAG RED RETROSPOT),1
4,0.062277,(PARTY BUNTING),1
...,...,...,...
824,0.010001,(PACK OF 20 NAPKINS RED APPLES),1
825,0.010001,(WOODEN SKITTLES GARDEN SET),1
826,0.010001,(CHRISTMAS RETROSPOT STAR WOOD),1
827,0.010001,(PANTRY MAGNETIC SHOPPING LIST),1


# Find Association between frequent bought items
 From association rules, we could extract information and even discover knowledge about which items that is more effective to be sold together

In [30]:
# find the association between frequenty bought items
from mlxtend.frequent_patterns import association_rules

In [35]:
association_rules(frequent_items, metric='lift',
                  min_threshold=1).sort_values(['confidence','lift'], ascending=False).reset_index(drop=True)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(POPPY'S PLAYHOUSE LIVINGROOM ),(POPPY'S PLAYHOUSE KITCHEN),0.012761,0.017241,0.011397,0.893130,51.801527,0.011177,9.195813
1,"(ROSES REGENCY TEACUP AND SAUCER , PINK REGENC...",(GREEN REGENCY TEACUP AND SAUCER),0.017112,0.026852,0.015228,0.889943,33.142040,0.014769,8.842220
2,"(TOILET METAL SIGN, KITCHEN METAL SIGN)",(BATHROOM METAL SIGN),0.011429,0.029060,0.010098,0.883523,30.403054,0.009766,8.335872
3,"(REGENCY CAKESTAND 3 TIER, PINK REGENCY TEACUP...",(GREEN REGENCY TEACUP AND SAUCER),0.011949,0.026852,0.010520,0.880435,32.787945,0.010199,8.139053
4,"(REGENCY CAKESTAND 3 TIER, PINK REGENCY TEACUP...",(ROSES REGENCY TEACUP AND SAUCER ),0.011949,0.029775,0.010293,0.861413,28.931078,0.009937,7.000842
...,...,...,...,...,...,...,...,...,...
525,(WHITE HANGING HEART T-LIGHT HOLDER),(COOK WITH WINE METAL SIGN ),0.151016,0.037113,0.010715,0.070952,1.911806,0.005110,1.036424
526,(WHITE HANGING HEART T-LIGHT HOLDER),(WOOD BLACK BOARD ANT WHITE FINISH),0.151016,0.032177,0.010553,0.069877,2.171630,0.005693,1.040532
527,(WHITE HANGING HEART T-LIGHT HOLDER),(JUMBO STORAGE BAG SUKI),0.151016,0.052374,0.010455,0.069232,1.321897,0.002546,1.018113
528,(WHITE HANGING HEART T-LIGHT HOLDER),(SCOTTIE DOG HOT WATER BOTTLE),0.151016,0.044646,0.010423,0.069017,1.545890,0.003681,1.026178


# Insights
From the association_rules results above, we could see that POPPY'S PLAYHOUSE LIVINGROOM and POPPY'S PLAYHOUSE BEDROOM are the items that has the highest association each other since these two items has the highest “lift” value. The higher the lift value, the higher the association between the items will be. If the lift value is more than 1, it is enough for us to say that those two items are associated each other. In thise case, the highest value is 51.801527 which is very high. It means these 2 items are very good to be sold together.

From the confidence, we could even extract more information. Remember that the confidence value is influenced by the antecedent and consequent. If the antecedent is higher than the consequent, then the rule that will be applied is rule number 1 (not number 2). vice versa. In this case, the antecedent value is lower than the consequent value. It means we will apply rule number 2 which is POPPY'S PLAYHOUSE BEDROOM → POPPY'S PLAYHOUSE LIVINGROOM. In a more detail explanation, it means that a customer will tends to bought poppy's playhouse livingromm AFTER they bought poppy's playhouse bedroom. Not in the other way around. This could be a very valuable information, because we are now aware which products should we put the discounts on. We could give a discounts on poppy's playhouse livingrommif a customer buy poppy's playhouse bedroom